This notebook includes a PyQt6 interface that generates visual explanation images of the model’s predictions and compiles them into an HTML page.

In [1]:
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QLabel, QPushButton, QFileDialog, 
    QVBoxLayout, QHBoxLayout, QWidget, QProgressBar, QTextEdit,
    QTableView, QSplitter, QTabWidget, QComboBox, QFrame
)
from PyQt6.QtCore import Qt, QThread, pyqtSignal, QSize
from PyQt6.QtGui import QStandardItemModel, QStandardItem, QPalette, QColor
import sys
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Layer, Dense, Input, LSTM, Bidirectional, Dropout, concatenate
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from lime import lime_tabular
from datetime import datetime
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
class SelfAttention(Layer):
    def __init__(self, attention_units=128, return_attention=False, **kwargs):
        self.attention_units = attention_units
        self.return_attention = return_attention
        super(SelfAttention, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.time_steps = input_shape[1]
        self.input_dim = input_shape[2]

        self.query_dense = Dense(self.attention_units)
        self.key_dense = Dense(self.attention_units)
        self.value_dense = Dense(self.input_dim)  
        
        self.context_dense = Dense(self.input_dim)
        
        super(SelfAttention, self).build(input_shape)
    
    def call(self, inputs):
        query = self.query_dense(inputs)  
        key = self.key_dense(inputs)     
        value = self.value_dense(inputs)  
        
        score = tf.matmul(query, key, transpose_b=True) 
        score = score / tf.math.sqrt(tf.cast(self.attention_units, tf.float32))

        attention_weights = tf.nn.softmax(score, axis=-1)  
        
        context = tf.matmul(attention_weights, value)  
        
        output = self.context_dense(context)  
        
        if self.return_attention:
            return output, attention_weights
        return output
    
    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[1], self.input_dim), 
                    (input_shape[0], input_shape[1], input_shape[1])]
        return (input_shape[0], input_shape[1], self.input_dim)
    
    def get_config(self):
        config = super(SelfAttention, self).get_config()
        config.update({
            'attention_units': self.attention_units,
            'return_attention': self.return_attention
        })
        return config

In [3]:
class DataTableModel(QStandardItemModel):
    def __init__(self):
        super().__init__()
        
    def load_data(self, data):
        self.clear()
        self.setHorizontalHeaderLabels(data.columns)
        
        # Only show first 1000 rows for performance
        display_data = data.head(1000)
        for row in range(len(display_data)):
            items = [QStandardItem(str(display_data.iloc[row, col])) for col in range(len(display_data.columns))]
            self.appendRow(items)

class StyledProgressBar(QProgressBar):
    def __init__(self):
        super().__init__()
        self.setStyleSheet("""
            QProgressBar {
                border: 2px solid grey;
                border-radius: 5px;
                text-align: center;
            }
            QProgressBar::chunk {
                background-color: #4CAF50;
                width: 10px;
                margin: 0.5px;
            }
        """)

class StyledButton(QPushButton):
    def __init__(self, text):
        super().__init__(text)
        self.setStyleSheet("""
            QPushButton {
                background-color: #4CAF50;
                color: white;
                border: none;
                padding: 8px 16px;
                border-radius: 4px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #45a049;
            }
            QPushButton:disabled {
                background-color: #cccccc;
            }
        """)

In [4]:
class DataPreprocessingPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop=None):
        self.columns_to_drop = columns_to_drop if columns_to_drop is not None else [
            '427_0', '167_5', '272_0', '272_1', '272_4', '835_0', '291_1', '291_2', 
            '291_6', '291_7', '291_8', '291_9', '291_10', '158_1', '158_2', '158_3', 
            '158_4', '158_5', '158_6', '158_7', '158_8', '459_3', '459_4', '459_5', 
            '459_6', '459_7', '459_8', '459_9', '459_10', '459_11', '459_12', '397_1', 
            '397_2', '397_13', '397_14', '397_16', '397_19', '397_20', '397_21', 
            '397_26', '397_28', '397_29', '397_32', '397_33'
        ]
        self.scaler = StandardScaler()
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, data):
        df = data.copy()
        
        # Drop specified columns if they exist
        existing_columns = [col for col in self.columns_to_drop if col in df.columns]
        df = df.drop(columns=existing_columns)
        
        df = df.groupby('vehicle_id').apply(self._handle_missing_values).reset_index(drop=True)
        
        return df
    
    def _handle_missing_values(self, group):
        # Forward fill
        group = group.fillna(method='ffill')
        # Backward fill remaining missing values
        group = group.fillna(method='bfill')
        # Fill any remaining missing values with 0
        group = group.fillna(0)
        return group


In [5]:
class LIMEReportThread(QThread):
    progress = pyqtSignal(int)
    status = pyqtSignal(str)
    finished = pyqtSignal(str)
    error = pyqtSignal(str)

    def __init__(self, data_path, model_path, num_instances):
        super().__init__()
        self.data_path = data_path
        self.model_path = model_path
        self.requested_instances = num_instances
        # Try to load the label encoder from the same directory as the model
        self.label_encoder_path = os.path.join(os.path.dirname(model_path), 'label_encoder.joblib')
        # Path to the saved scaler
        self.scaler_path = os.path.join(os.path.dirname(model_path), 'standard_scaler.joblib')
        self.pipeline = DataPreprocessingPipeline()

    def weighted_loss(self, y_true, y_pred):
        cost_matrix = tf.constant([
            [0, 7, 8, 9, 10],
            [200, 0, 7, 8, 9],
            [300, 200, 0, 7, 8],
            [400, 300, 200, 0, 7],
            [500, 400, 300, 200, 0]
        ], dtype=tf.float32)
        
        y_true_onehot = tf.one_hot(tf.cast(tf.squeeze(y_true), tf.int32), depth=5)
        cost_weighted_pred = tf.matmul(y_pred, tf.transpose(cost_matrix))
        weighted_output = tf.reduce_sum(y_true_onehot * cost_weighted_pred, axis=1)
        base_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
        return tf.reduce_mean(base_loss * weighted_output)

    def custom_cost_metric(self, y_true, y_pred):
        # Convert predictions to probabilities
        y_pred = tf.nn.softmax(y_pred)
        
        # Get the predicted class
        pred_class = tf.argmax(y_pred, axis=1)
        true_class = tf.argmax(y_true, axis=1)
        
        # Define cost matrix
        cost_matrix = tf.constant([
            [0, 7, 8, 9, 10],
            [200, 0, 7, 8, 9],
            [300, 200, 0, 7, 8],
            [400, 300, 200, 0, 7],
            [500, 400, 300, 200, 0]
        ], dtype=tf.float32)
        
        # Calculate cost
        costs = tf.gather_nd(cost_matrix, 
                            tf.stack([true_class, pred_class], axis=1))
        return tf.reduce_mean(costs)

    def preprocess_test_data(self, data):
        X, vehicle_ids = [], []
        grouped = data.groupby('vehicle_id')
        
        # Load the pre-trained scaler
        try:
            scaler = joblib.load(self.scaler_path)
            self.status.emit("Pre-trained scaler loaded successfully")
        except Exception as e:
            self.error.emit(f"Error loading scaler: {str(e)}")
            return None, None, None
        
        # Find max length in this batch
        max_length = max(len(group) for _, group in grouped)
        
        for vehicle_id, group in grouped:
            time_series = group.sort_values('time_step').iloc[:, 2:].values
            time_series = scaler.transform(time_series)
            X.append(time_series)
            vehicle_ids.append(vehicle_id)
        
        # Pad sequences to max length in batch
        X_padded = pad_sequences(X, maxlen=max_length, padding='post', dtype='float32')
        return X_padded, vehicle_ids, max_length

    def create_attention_model(self, original_model):
        """Create a model that returns attention weights"""
        # Find the attention layer in the model
        attention_layer = None
        attention_layer_index = -1
        
        for i, layer in enumerate(original_model.layers):
            if isinstance(layer, SelfAttention):
                attention_layer = layer
                attention_layer_index = i
                break
        
        if attention_layer is None:
            return None, None
        
        # Create new attention layer that returns weights
        new_attention = SelfAttention(
            attention_units=attention_layer.attention_units,
            return_attention=True
        )
        
        # Safer approach: Get the input to the attention layer
        if attention_layer_index > 0:
            # Get the output of the layer before attention
            pre_attention_model = Model(
                inputs=original_model.input,
                outputs=original_model.layers[attention_layer_index - 1].output
            )
            
            # Create a simple model just for attention weights
            attention_input = Input(shape=pre_attention_model.output_shape[1:])
            attention_output, attention_weights = new_attention(attention_input)
            simple_attention_model = Model(inputs=attention_input, outputs=[attention_output, attention_weights])
            
            # Copy weights to new attention layer
            new_attention.set_weights(attention_layer.get_weights())
            
            return (pre_attention_model, simple_attention_model), True
        
        return None, None

    def visualize_attention(self, instance_idx, X_test, predictions, vehicle_ids, label_encoder, results_dir, model):
        """Generate attention visualization for a single instance"""
        # Create attention model
        models, has_attention = self.create_attention_model(model)
        
        if not has_attention:
            self.status.emit("Model does not have attention layer, skipping attention visualization")
            return None
        
        pre_attention_model, attention_model = models
        
        # Get attention weights for this instance
        instance_data = X_test[instance_idx:instance_idx+1]
        
        # First, get the pre-attention features
        pre_attention_features = pre_attention_model.predict(instance_data, verbose=0)
        
        # Then get attention weights
        _, attention_weights = attention_model.predict(pre_attention_features, verbose=0)
        
        # Get predictions
        probs = tf.nn.softmax(predictions[instance_idx]).numpy()
        predicted_class = np.argmax(probs)
        
        plt.figure(figsize=(15, 10))
        
        # First subplot: Attention Heatmap
        plt.subplot(2, 1, 1)
        
        # Find non-zero time steps (remove padding)
        non_zero_steps = np.any(X_test[instance_idx] != 0, axis=1)
        actual_length = np.sum(non_zero_steps)
        
        # Get relevant part of attention weights
        relevant_attention = attention_weights[0][:actual_length, :actual_length]
        
        # Show heatmap
        sns.heatmap(relevant_attention, cmap='viridis')
        plt.title(f'Attention Weights for Vehicle {vehicle_ids[instance_idx]}')
        plt.xlabel('Time Steps')
        plt.ylabel('Time Steps')
        
        # Second subplot: Prediction probabilities
        plt.subplot(2, 1, 2)
        class_names = label_encoder.classes_
        
        # Prediction probabilities
        bars = plt.bar(class_names, probs)
        
        # Highlight predicted class
        for j, bar in enumerate(bars):
            if j == predicted_class:
                bar.set_color('green')
        
        plt.axhline(y=0.2, color='gray', linestyle='--')  # Reference line
        plt.title(f'Class Predictions: Predicted={class_names[predicted_class]}')
        plt.ylabel('Probability')
        plt.ylim(0, 1)
        
        # Add percentage labels on bars
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height*100:.1f}%',
                    ha='center', va='bottom')
        
        plt.tight_layout()
        attention_filename = f"attention_visualization_vehicle_{vehicle_ids[instance_idx]}.png"
        plt.savefig(os.path.join(results_dir, attention_filename))
        plt.close()
        
        return attention_filename

    def create_feature_importance_plots_for_vehicle(self, instance_idx, X_test, vehicle_ids, label_encoder, results_dir, model):
        """Generate feature importance plots for a specific vehicle"""
        # Create attention model
        models, has_attention = self.create_attention_model(model)
        
        if not has_attention:
            self.status.emit("Model does not have attention layer, skipping feature importance plots")
            return None, None
        
        pre_attention_model, attention_model = models
        
        # Get attention weights for this specific instance
        instance_data = X_test[instance_idx:instance_idx+1]
        pre_features = pre_attention_model.predict(instance_data)
        _, attention_weights = attention_model.predict(pre_features)
        
        # Find actual length (non-padding)
        actual_length = np.sum(np.any(X_test[instance_idx] != 0, axis=1))
        
        # Attention matrix for this example
        att_matrix = attention_weights[0][:actual_length, :actual_length]
        
        # Time step importance - use column means instead of row sums
        # This gives us how much each time step is attended to
        step_importance = np.mean(att_matrix, axis=0)
        
        # Alternative calculation: max attention per time step
        max_attention = np.max(att_matrix, axis=0)
        
        # Combine both metrics for more robust importance scores
        combined_importance = (step_importance + max_attention) / 2
        
        # Add variance to capture attention spread
        attention_variance = np.var(att_matrix, axis=0)
        
        # Create figure with subplots
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
        
        # Plot 1: Combined importance
        ax1.plot(range(len(combined_importance)), combined_importance, marker='o', linewidth=2, markersize=8, label='Combined')
        ax1.plot(range(len(step_importance)), step_importance, marker='s', linewidth=1.5, markersize=6, alpha=0.7, label='Mean Attention')
        ax1.plot(range(len(max_attention)), max_attention, marker='^', linewidth=1.5, markersize=6, alpha=0.7, label='Max Attention')
        
        ax1.set_title(f'Feature Importance Across Time Steps - Vehicle {vehicle_ids[instance_idx]}', fontsize=16)
        ax1.set_xlabel('Time Steps', fontsize=14)
        ax1.set_ylabel('Importance Score', fontsize=14)
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # Add importance values as text on the plot for top values
        top_indices = np.argsort(combined_importance)[-10:]  # Top 10 time steps
        for idx in top_indices:
            if idx < len(combined_importance):
                ax1.text(idx, combined_importance[idx] + 0.01, f'{combined_importance[idx]:.3f}', 
                        ha='center', va='bottom', fontsize=8, rotation=45)
        
        # Plot 2: Attention variance (shows which time steps have more varied attention)
        ax2.bar(range(len(attention_variance)), attention_variance, alpha=0.7, color='darkblue')
        ax2.set_title('Attention Variance per Time Step', fontsize=14)
        ax2.set_xlabel('Time Steps', fontsize=14)
        ax2.set_ylabel('Variance', fontsize=14)
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        feature_importance_filename = f'feature_importance_vehicle_{vehicle_ids[instance_idx]}.png'
        plt.savefig(os.path.join(results_dir, feature_importance_filename))
        plt.close()
        
        return feature_importance_filename, None

    def create_lime_explanation(self, instance_idx, explanation, time_series_length, X_test, predictions, results_dir, vehicle_ids, feature_names):
        weights_df = pd.DataFrame(explanation.as_list(), columns=["Feature", "Weight"])
        weights_df['base_feature'] = weights_df['Feature'].apply(lambda x: '_'.join(x.split('_')[1:3]) if len(x.split('_')) >= 4 else x)
        weights_df['abs_weight'] = weights_df['Weight'].abs()
        top_20_weights = weights_df.nlargest(20, 'abs_weight')
        weights_df = weights_df.drop('abs_weight', axis=1)
        top_20_weights = top_20_weights.drop('abs_weight', axis=1)
        unique_base_features = sorted(top_20_weights['base_feature'].unique())
        time_series_data = X_test[instance_idx]

        # Generate plots
        plt.figure(figsize=(20, 12))
        top_20_sorted = top_20_weights.sort_values('Weight', ascending=True)
        colors = ['red' if w < 0 else 'green' for w in top_20_sorted['Weight']]
        bars = plt.barh(range(len(top_20_sorted)), top_20_sorted['Weight'], color=colors)
        plt.yticks(range(len(top_20_sorted)), top_20_sorted['Feature'], fontsize=10)
        plt.xlabel("Impact on Prediction (Weight)", fontsize=14)
        plt.title(f"Top 20 Most Influential Features for Instance {vehicle_ids[instance_idx]}", fontsize=18)
        plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
        
        for i, bar in enumerate(bars):
            width = bar.get_width()
            plt.text(width, bar.get_y() + bar.get_height()/2,
                    f'{width:.3f}',
                    ha='left' if width >= 0 else 'right',
                    va='center',
                    fontsize=10)
        
        feature_plot_filename = f"feature_importance_{vehicle_ids[instance_idx]}.png"
        plt.tight_layout()
        plt.savefig(os.path.join(results_dir, feature_plot_filename))
        plt.close()

        # Create time series visualizations
        feature_indices = {}
        for i, feature in enumerate(feature_names):
            base_feature = feature
            if '_' in feature:
                parts = feature.split('_')
                if len(parts) >= 2:
                    base_feature = f"{parts[0]}_{parts[1]}"
            feature_indices[base_feature] = i

        plt.figure(figsize=(20, 10))
        for base_feature in unique_base_features:
            if base_feature in feature_indices:
                feature_idx = feature_indices[base_feature]
                plt.plot(range(time_series_length), 
                        time_series_data[:time_series_length, feature_idx], 
                        label=f'Feature {base_feature}')
        
        plt.title('Time Series of Selected Features (From Top 20)', fontsize=14)
        plt.xlabel('Time Step')
        plt.ylabel('Normalized Value')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        timeseries_plot_filename = f"timeseries_{vehicle_ids[instance_idx]}.png"
        plt.tight_layout()
        plt.savefig(os.path.join(results_dir, timeseries_plot_filename), bbox_inches='tight')
        plt.close()

        # Feature correlation plot
        plt.figure(figsize=(12, 10))
        selected_data = {}
        for base_feature in unique_base_features:
            if base_feature in feature_indices:
                feature_idx = feature_indices[base_feature]
                selected_data[base_feature] = time_series_data[:time_series_length, feature_idx]
        
        if selected_data:
            correlation_matrix = pd.DataFrame(selected_data).corr()
            sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
            plt.title('Feature Correlation Heatmap (From Top 20 Features)', fontsize=14)
        correlation_plot_filename = f"correlation_{vehicle_ids[instance_idx]}.png"
        plt.tight_layout()
        plt.savefig(os.path.join(results_dir, correlation_plot_filename))
        plt.close()

        # Feature distribution plot
        plt.figure(figsize=(14, 8))
        for base_feature in unique_base_features:
            if base_feature in feature_indices:
                feature_idx = feature_indices[base_feature]
                sns.kdeplot(time_series_data[:time_series_length, feature_idx], 
                           label=f'Feature {base_feature}')
        
        plt.title('Feature Value Distributions (From Top 20 Features)', fontsize=14)
        plt.xlabel('Normalized Value')
        plt.ylabel('Density')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        distribution_plot_filename = f"distribution_{vehicle_ids[instance_idx]}.png"
        plt.tight_layout()
        plt.savefig(os.path.join(results_dir, distribution_plot_filename), bbox_inches='tight')
        plt.close()

        return {
            'feature': feature_plot_filename,
            'timeseries': timeseries_plot_filename,
            'correlation': correlation_plot_filename,
            'distribution': distribution_plot_filename
        }, top_20_weights

    def create_detailed_report(self, instance_idx, explanation, predictions, feature_names, label_encoder, plot_files, weights_df, results_dir, attention_plot, feature_importance_plots):
        predicted_class = np.argmax(predictions[instance_idx])
        messages = {
            0: "The truck is not expected to fail within the next 48 hours.",
            1: "The truck is expected to fail within 24-48 hours.",
            2: "The truck is expected to fail within 12-24 hours.",
            3: "The truck is expected to fail within 6-12 hours.",
            4: "The truck is expected to fail within 0-6 hours."
        }
        prediction_message = messages[predicted_class]
        
        # Check if visualizations directory exists and get visualization images
        visualizations_dir = os.path.join(os.getcwd(), "visualizations")
        viz_images = []
        viz_descriptions = {
            "permutation_feature_importance_bar.png": {
                "title": "Permutation Feature Importance (Bar Chart)",
                "description": "This bar chart shows the top 20 most important features determined by GPU-accelerated permutation. The importance score indicates how much the model's performance degrades when that feature's values are randomly shuffled. Higher scores indicate more critical features for the model's predictions."
            },
            "permutation_importance_heatmap.png": {
                "title": "Feature Importance Heatmap by Class",
                "description": "This heatmap displays feature importance scores across all prediction classes (0-4). The color intensity represents the importance level for each feature-class combination. Darker regions indicate higher importance, helping identify which features are most relevant for specific failure time windows."
            },
            "class_specific_feature_importance.png": {
                "title": "Class-Specific Feature Importance Analysis",
                "description": "These subplots show the top 15 features for each prediction class (0-4 representing different failure time windows). Each class has its own importance ranking, revealing which features are most predictive for specific failure scenarios. This analysis helps understand the model's decision-making process for different risk levels."
            },
            "feature_importance_variance.png": {
                "title": "Feature Importance Variance Analysis",
                "description": "This chart shows features with the highest variance in importance across different classes. High variance indicates that a feature's importance changes significantly depending on the prediction class, suggesting these features are particularly discriminative for distinguishing between different failure time windows."
            }
        }
        
        if os.path.exists(visualizations_dir):
            for image_name in ["permutation_feature_importance_bar.png", "permutation_importance_heatmap.png", 
                              "class_specific_feature_importance.png", "feature_importance_variance.png"]:
                image_path = os.path.join(visualizations_dir, image_name)
                if os.path.exists(image_path):
                    # Use relative path for HTML - The visualizations folder is two levels above the batch directory
                    relative_path = f"../../../visualizations/{image_name}"
                    viz_images.append({
                        "path": relative_path,
                        "title": viz_descriptions[image_name]["title"],
                        "description": viz_descriptions[image_name]["description"]
                    })
        
        report_filename = f"explanation_report_instance_{instance_idx}.html"
        report_path = os.path.join(results_dir, report_filename)

        html_content = f"""
        <html>
        <head>
            <title>LIME Explanation Report - Instance {instance_idx}</title>
            <style>
                body {{ 
                    font-family: Arial, sans-serif; 
                    margin: 20px; 
                    text-align: center; 
                    background-color: #f5f5f5;
                }}
                .container {{ 
                    max-width: 1200px; 
                    margin: auto; 
                    background-color: white;
                    padding: 20px;
                    border-radius: 10px;
                    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
                }}
                .section {{ 
                    margin-bottom: 40px; 
                    display: none;
                }}
                .section.active {{
                    display: block;
                }}
                .analysis-subsection {{
                    margin-bottom: 30px;
                    padding: 20px;
                    border: 1px solid #ddd;
                    border-radius: 8px;
                    background-color: #fafafa;
                }}
                .feature-table {{ 
                    width: 100%; 
                    border-collapse: collapse; 
                    margin: auto; 
                }}
                .feature-table th, .feature-table td {{ 
                    border: 1px solid #ddd; 
                    padding: 8px; 
                    text-align: center; 
                }}
                .feature-table tr:nth-child(even) {{ 
                    background-color: #f2f2f2; 
                }}
                .prediction-box {{ 
                    padding: 15px; 
                    margin: 10px auto; 
                    border-radius: 5px; 
                    background-color: #f8f9fa; 
                    max-width: 800px; 
                }}
                .visualization-grid {{ 
                    display: grid; 
                    grid-template-columns: 1fr 1fr; 
                    gap: 20px; 
                    margin: 20px auto; 
                }}
                .visualization-item {{ 
                    border: 1px solid #ddd; 
                    padding: 15px; 
                    border-radius: 5px; 
                }}
                .explanation-text {{ 
                    background-color: #f8f9fa; 
                    padding: 15px; 
                    border-left: 4px solid #007bff;
                    margin: 10px 0;
                    font-size: 14px;
                    line-height: 1.5;
                    text-align: left;
                }}
                .centered-image {{ 
                    display: block; 
                    margin: auto; 
                    max-width: 100%; 
                    height: auto;
                }}
                h1, h2, h3 {{ 
                    text-align: center; 
                }}
                .navigation-buttons {{
                    text-align: center;
                    margin: 20px 0;
                    padding: 20px;
                    background-color: #e9ecef;
                    border-radius: 8px;
                }}
                .nav-btn {{
                    background-color: #007bff;
                    color: white;
                    border: none;
                    padding: 12px 24px;
                    margin: 0 10px;
                    border-radius: 5px;
                    cursor: pointer;
                    font-size: 16px;
                    font-weight: bold;
                    transition: background-color 0.3s;
                }}
                .nav-btn:hover {{
                    background-color: #0056b3;
                }}
                .nav-btn.active {{
                    background-color: #28a745;
                }}
                .nav-btn:disabled {{
                    background-color: #6c757d;
                    cursor: not-allowed;
                }}
                .viz-section {{
                    margin-bottom: 30px;
                    padding: 20px;
                    border: 1px solid #ddd;
                    border-radius: 8px;
                    background-color: #fafafa;
                }}
            </style>
            <script>
                function showSection(sectionId) {{
                    // Hide all sections
                    var sections = document.querySelectorAll('.section');
                    sections.forEach(function(section) {{
                        section.classList.remove('active');
                    }});
                    
                    // Show selected section
                    document.getElementById(sectionId).classList.add('active');
                    
                    // Update button states
                    var buttons = document.querySelectorAll('.nav-btn');
                    buttons.forEach(function(btn) {{
                        btn.classList.remove('active');
                    }});
                    
                    // Activate current button
                    if (sectionId === 'main-analysis') {{
                        document.getElementById('main-btn').classList.add('active');
                    }} else {{
                        document.getElementById('viz-btn').classList.add('active');
                    }}
                }}
                
                window.onload = function() {{
                    showSection('main-analysis');
                }}
            </script>
        </head>
        <body>
            <div class="container">
                <h1>LIME Explanation Report - Instance {instance_idx}</h1>
                
                <div class="navigation-buttons">
                    <button id="main-btn" class="nav-btn active" onclick="showSection('main-analysis')">
                        Main Analysis
                    </button>
                    <button id="viz-btn" class="nav-btn" onclick="showSection('visualizations')">
                        Global Feature Analysis
                    </button>
                </div>

                <!-- Main Analysis Section -->
                <div id="main-analysis" class="section active">
                    <div class="analysis-subsection">
                        <h2>Prediction Summary</h2>
                        <div class="prediction-box">
                            <p><strong>Instance ID:</strong> {instance_idx}</p>
                            <p><strong>Predicted Class:</strong> {label_encoder.inverse_transform([predicted_class])[0]}</p>
                            <p><strong>Prediction Message:</strong> {prediction_message}</p>
        """
        
        if attention_plot:
            html_content += f"""
                            <img src="{attention_plot}" class="centered-image" style="max-width: 800px;">
                            <div class="explanation-text">
                                <strong>Attention Visualization:</strong> The top heatmap shows the attention weights across time steps, 
                                indicating which time points the model focuses on when making predictions. Brighter colors indicate 
                                higher attention. The bottom graph shows the predicted probabilities for each class, with the green 
                                bar indicating the model's final prediction.
                            </div>
            """
        
        html_content += f"""
                        </div>
                    </div>

                    <div class="analysis-subsection">
                        <h2>Feature Importance Analysis</h2>
                        <img src="{plot_files['feature']}" class="centered-image">
                        <div class="explanation-text">
                            <strong>Feature Importance Graph:</strong> This graph shows the most important features identified by LIME
                            and their impact on the prediction. Positive values (bars to the right) show features supporting the prediction,
                            while negative values (bars to the left) show features opposing the prediction. Bar length represents the
                            feature's importance.
                        </div>
                    </div>

                    <div class="analysis-subsection">
                        <h2>Feature Weights Detail</h2>
                        <table class="feature-table">
                            <tr>
                                <th>Feature</th>
                                <th>Weight</th>
                                <th>Impact</th>
                            </tr>
        """
        
        for _, row in weights_df.iterrows():
            impact = "Positive" if row['Weight'] > 0 else "Negative"
            color = "green" if row['Weight'] > 0 else "red"
            html_content += f"""
                            <tr>
                                <td>{row['Feature']}</td>
                                <td style="color: {color}">{row['Weight']:.4f}</td>
                                <td>{impact}</td>
                            </tr>
            """

        html_content += f"""
                        </table>
                        <div class="explanation-text">
                            <strong>Feature Weights Table:</strong> This table shows the numerical weights and impacts of features
                            determined by LIME in detail. Positive weights (green) show features supporting the prediction,
                            while negative weights (red) show features opposing the prediction.
                        </div>
                    </div>

                    <div class="analysis-subsection">
                        <h2>Temporal Feature Importance Analysis</h2>
        """
        
        if feature_importance_plots[0]:
            html_content += f"""
                        <img src="{feature_importance_plots[0]}" class="centered-image">
                        <div class="explanation-text">
                            <strong>Feature Importance Across Time Steps:</strong> This graph shows which time steps in the 
                            sequence are most important for this specific vehicle's prediction. Higher importance values indicate 
                            that the model pays more attention to those specific time points when making decisions. The markers 
                            show the exact importance values for each time step.
                        </div>
            """
        else:
            html_content += """
                        <p style="text-align: center; color: #666;">Attention-based feature importance analysis not available for this model.</p>
            """
        
        html_content += f"""
                    </div>

                    <div class="analysis-subsection">
                        <h2>Time Series Analysis</h2>
                        <img src="{plot_files['timeseries']}" class="centered-image">
                        <div class="explanation-text">
                            <strong>Time Series Graph:</strong> This graph shows the changes in selected features over time.
                            Each line represents a different feature and allows us to track how values change.
                            This graph helps us understand anomalies, trends, and relationships between features in the time dimension.
                        </div>
                    </div>

                    <div class="visualization-grid">
                        <div class="visualization-item">
                            <h3>Feature Correlation Analysis</h3>
                            <img src="{plot_files['correlation']}" class="centered-image">
                            <div class="explanation-text">
                                <strong>Correlation Heatmap:</strong> This heatmap shows the strength of relationships between features.
                                Dark blue colors indicate strong positive correlation, while dark red colors indicate strong negative correlation.
                                This visualization helps us understand which features move together or in opposite directions.
                            </div>
                        </div>
                        <div class="visualization-item">
                            <h3>Feature Value Distributions</h3>
                            <img src="{plot_files['distribution']}" class="centered-image">
                            <div class="explanation-text">
                                <strong>Feature Distribution Graph:</strong> This graph shows how the values of each feature are distributed.
                                The shape of the distribution provides information about central tendency and spread.
                                This information helps us understand whether features follow a normal distribution and identify the presence
                                of outliers.
                            </div>
                        </div>
                    </div>
                </div>

                <!-- Visualizations Section -->
                <div id="visualizations" class="section">
                    <h2>Global Feature Analysis & Model Insights</h2>
                    <div class="explanation-text">
                        <strong>Global Analysis Overview:</strong> This section contains comprehensive visualizations that analyze 
                        feature importance patterns across the entire dataset and all prediction classes. These visualizations 
                        provide insights into the overall model behavior and feature relationships beyond individual predictions.
                    </div>
        """
        
        # Add visualization images if they exist
        for viz in viz_images:
            html_content += f"""
                    <div class="viz-section">
                        <h3>{viz['title']}</h3>
                        <img src="{viz['path']}" class="centered-image">
                        <div class="explanation-text">
                            {viz['description']}
                        </div>
                    </div>
            """
        
        if not viz_images:
            html_content += """
                    <div class="viz-section">
                        <p style="text-align: center; color: #666; font-style: italic;">
                            Global visualization files not found in the 'visualizations' directory. 
                            Please ensure the following files exist in the visualizations folder:
                        </p>
                        <ul style="text-align: left; max-width: 600px; margin: auto;">
                            <li>permutation_feature_importance_bar.png</li>
                            <li>permutation_importance_heatmap.png</li>
                            <li>class_specific_feature_importance.png</li>
                            <li>feature_importance_variance.png</li>
                        </ul>
                    </div>
            """
        
        html_content += """
                </div>
            </div>
        </body>
        </html>
        """
        
        with open(report_path, "w", encoding="utf-8") as f:
            f.write(html_content)
        return report_path

    def run(self):
        try:
            self.status.emit("Loading data and model...")
            self.progress.emit(10)
    
            self.model_path = os.path.normpath(self.model_path)
            if not os.path.exists(self.model_path):
                raise FileNotFoundError(f"Model file not found at: {self.model_path}")
    
            # Create directories
            main_dir = "lime_explanations"
            if not os.path.exists(main_dir):
                os.makedirs(main_dir)
    
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            batch_dir = os.path.join(main_dir, f"batch_{timestamp}")
            os.makedirs(batch_dir)
    
            # Load and preprocess data
            self.status.emit("Loading dataset...")
            data = pd.read_csv(self.data_path)
            processed_data = self.pipeline.transform(data)
            feature_names = processed_data.columns[2:].tolist()
            self.status.emit("Dataset loaded successfully")
    
            # Load model with custom objects
            self.status.emit("Loading model...")
            custom_objects = {
                'weighted_loss': self.weighted_loss,
                'custom_cost_metric': self.custom_cost_metric,
                'SelfAttention' : SelfAttention
            }
            try:
                model = tf.keras.models.load_model(self.model_path, custom_objects=custom_objects)
                self.status.emit("Model loaded successfully")
            except Exception as e:
                # HDF5 fallback
                self.status.emit(f"Failed to load as .keras: {str(e)}. Trying as HDF5...")
                try:
                    model = tf.keras.models.load_model(self.model_path, custom_objects=custom_objects)
                    self.status.emit("Model loaded successfully as HDF5")
                except Exception as e2:
                    self.error.emit(f"Failed to load model: {str(e2)}")
                    return
    
            # Load label encoder
            try:
                label_encoder = joblib.load(self.label_encoder_path)
                self.status.emit("Label encoder loaded successfully")
            except Exception as e:
                self.error.emit(f"Error loading label encoder: {str(e)}")
                return
    
            self.status.emit("Preprocessing data...")
            self.progress.emit(30)
    
            # Use the new preprocess_test_data method directly from this class
            X_test, vehicle_ids, max_seq_length = self.preprocess_test_data(processed_data)
            if X_test is None:
                self.error.emit("Failed to preprocess data with saved scaler")
                return
            
            # Make predictions once for all data to avoid TF retracing warning
            self.status.emit("Making predictions...")
            predictions = model.predict(X_test, batch_size=32, verbose=0)
    
            # LIME setup and report generation
            self.status.emit("Setting up LIME explainer...")
            self.progress.emit(50)
    
            total_instances = min(self.requested_instances, len(vehicle_ids))
            self.status.emit(f"Processing {total_instances} instances (requested: {self.requested_instances})")
    
            def create_feature_names(feature_names, timesteps):
                return [f"feature_{feat}_{t}" for feat in feature_names for t in range(timesteps)]
    
            formatted_feature_names = create_feature_names(feature_names, max_seq_length)
    
            lime_explainer = lime_tabular.LimeTabularExplainer(
                X_test.reshape(X_test.shape[0], -1),
                feature_names=formatted_feature_names,
                class_names=label_encoder.classes_,
                mode='classification',
                discretize_continuous=False,
            )
    
            # Create a prediction function that uses fixed batch size
            def lime_predict(data):
                reshaped_data = data.reshape(data.shape[0], X_test.shape[1], X_test.shape[2])
                
                # Always use a fixed batch size to avoid retracing
                batch_size = 32
                
                if reshaped_data.shape[0] <= batch_size:
                    # Pad to exact batch size
                    pad_size = batch_size - reshaped_data.shape[0]
                    if pad_size > 0:
                        padding = np.zeros((pad_size, X_test.shape[1], X_test.shape[2]))
                        padded_data = np.concatenate([reshaped_data, padding], axis=0)
                    else:
                        padded_data = reshaped_data
                    
                    # Always predict with exact batch size
                    predictions = model.predict(padded_data, batch_size=batch_size, verbose=0)
                    return predictions[:reshaped_data.shape[0]]
                else:
                    # For larger batches, process in chunks of exact batch size
                    predictions = []
                    for i in range(0, reshaped_data.shape[0], batch_size):
                        chunk = reshaped_data[i:i+batch_size]
                        if chunk.shape[0] < batch_size:
                            # Pad the last chunk to exact batch size
                            pad_size = batch_size - chunk.shape[0]
                            padding = np.zeros((pad_size, X_test.shape[1], X_test.shape[2]))
                            padded_chunk = np.concatenate([chunk, padding], axis=0)
                        else:
                            padded_chunk = chunk
                        
                        chunk_pred = model.predict(padded_chunk, batch_size=batch_size, verbose=0)
                        predictions.append(chunk_pred[:chunk.shape[0]])
                    
                    return np.concatenate(predictions, axis=0)
            
            # Create attention models once if available
            attention_models = None
            pre_attention_model = None
            attention_model = None
            try:
                attention_models = self.create_attention_model(model)
                if attention_models and attention_models[1]:
                    pre_attention_model, attention_model = attention_models[0]
                    # Compile models to optimize performance
                    pre_attention_model.compile(optimizer='adam', loss='mse')
                    attention_model.compile(optimizer='adam', loss='mse')
                    self.status.emit("Attention analysis enabled")
            except:
                self.status.emit("Attention analysis not available")
    
            self.status.emit("Generating explanations and reports...")
            
            for i, vehicle_id in enumerate(vehicle_ids[:total_instances]):
                progress = 50 + (i + 1) * 40 // total_instances
                self.progress.emit(progress)
                self.status.emit(f"Processing instance {i+1}/{total_instances}")
    
                instance_dir = os.path.join(batch_dir, f"vehicle_{vehicle_id}")
                os.makedirs(instance_dir, exist_ok=True)
    
                original_length = len(data[data['vehicle_id'] == vehicle_id])
                instance_data = X_test[i].flatten()
                explanation = lime_explainer.explain_instance(
                    instance_data,
                    lime_predict,
                    num_features=len(formatted_feature_names)
                )
    
                plot_files, weights_df = self.create_lime_explanation(
                    i, explanation, original_length, X_test, predictions,
                    instance_dir, vehicle_ids, feature_names
                )
                
                # Generate attention visualization
                attention_plot = self.visualize_attention(
                    i, X_test, predictions, vehicle_ids, label_encoder, instance_dir, model
                )
                
                # Generate feature importance plots for this specific vehicle
                feature_importance_plots = self.create_feature_importance_plots_for_vehicle(
                    i, X_test, vehicle_ids, label_encoder, instance_dir, model
                )
    
                report_path = self.create_detailed_report(
                    i, explanation, predictions, feature_names,
                    label_encoder, plot_files, weights_df, instance_dir,
                    attention_plot, feature_importance_plots
                )
    
            self.progress.emit(100)
            self.status.emit("Reports generated successfully!")
            self.finished.emit(batch_dir)
    
        except Exception as e:
            self.error.emit(f"Unexpected error: {str(e)}")

In [6]:
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("LIME Explanation Report Generator")
        self.setGeometry(100, 100, 1400, 900)
        
        self.data_path = None
        self.model_path = None
        
        self.setup_dark_theme()
        self.init_ui()

    def setup_dark_theme(self):
        dark_palette = QPalette()
        dark_palette.setColor(QPalette.ColorRole.Window, QColor(53, 53, 53))
        dark_palette.setColor(QPalette.ColorRole.WindowText, QColor(255, 255, 255))
        dark_palette.setColor(QPalette.ColorRole.Base, QColor(35, 35, 35))
        dark_palette.setColor(QPalette.ColorRole.AlternateBase, QColor(53, 53, 53))
        dark_palette.setColor(QPalette.ColorRole.ToolTipBase, QColor(255, 255, 255))
        dark_palette.setColor(QPalette.ColorRole.ToolTipText, QColor(255, 255, 255))
        dark_palette.setColor(QPalette.ColorRole.Text, QColor(255, 255, 255))
        dark_palette.setColor(QPalette.ColorRole.Button, QColor(53, 53, 53))
        dark_palette.setColor(QPalette.ColorRole.ButtonText, QColor(255, 255, 255))
        dark_palette.setColor(QPalette.ColorRole.BrightText, QColor(255, 0, 0))
        dark_palette.setColor(QPalette.ColorRole.Link, QColor(42, 130, 218))
        dark_palette.setColor(QPalette.ColorRole.Highlight, QColor(42, 130, 218))
        dark_palette.setColor(QPalette.ColorRole.HighlightedText, QColor(255, 255, 255))
        self.setPalette(dark_palette)

    def init_ui(self):
        main_widget = QWidget()
        main_layout = QVBoxLayout(main_widget)
        
        # Header section
        header_widget = QWidget()
        header_layout = QHBoxLayout(header_widget)
        
        # Title
        title_label = QLabel("LIME Explanation Report Generator")
        title_label.setStyleSheet("""
            font-size: 28px;
            font-weight: bold;
            color: #4CAF50;
            padding: 10px;
        """)
        title_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        header_layout.addWidget(title_label)
        
        main_layout.addWidget(header_widget)

        # Create splitter for tables and controls
        splitter = QSplitter(Qt.Orientation.Horizontal)
        
        # Left side - Table views with tabs
        self.tab_widget = QTabWidget()
        self.tab_widget.setStyleSheet("""
            QTabWidget::pane {
                border: 1px solid #444;
                background: #2d2d2d;
            }
            QTabBar::tab {
                background: #353535;
                color: #fff;
                padding: 8px 12px;
                border: 1px solid #444;
                border-bottom: none;
            }
            QTabBar::tab:selected {
                background: #4CAF50;
            }
        """)
        
        # Original data tab
        original_tab = QWidget()
        original_layout = QVBoxLayout(original_tab)
        self.original_table_view = QTableView()
        self.setup_table_view(self.original_table_view)
        self.original_table_model = DataTableModel()
        self.original_table_view.setModel(self.original_table_model)
        original_layout.addWidget(self.original_table_view)
        self.tab_widget.addTab(original_tab, "Original Data")
        
        # Processed data tab
        processed_tab = QWidget()
        processed_layout = QVBoxLayout(processed_tab)
        self.processed_table_view = QTableView()
        self.setup_table_view(self.processed_table_view)
        self.processed_table_model = DataTableModel()
        self.processed_table_view.setModel(self.processed_table_model)
        processed_layout.addWidget(self.processed_table_view)
        self.tab_widget.addTab(processed_tab, "Processed Data")
        
        # Add statistics labels with improved styling
        self.original_stats_label = QLabel()
        self.processed_stats_label = QLabel()
        self.setup_stats_label(self.original_stats_label)
        self.setup_stats_label(self.processed_stats_label)
        original_layout.addWidget(self.original_stats_label)
        processed_layout.addWidget(self.processed_stats_label)
        
        splitter.addWidget(self.tab_widget)
        
        # Right side - Controls
        controls_widget = QWidget()
        controls_layout = QVBoxLayout(controls_widget)
        controls_layout.setSpacing(15)
        
        # File selection section
        file_section = QFrame()
        file_section.setFrameStyle(QFrame.Shape.StyledPanel | QFrame.Shadow.Raised)
        file_layout = QVBoxLayout(file_section)
        
        # Dataset selection
        data_widget = QWidget()
        data_layout = QHBoxLayout(data_widget)
        self.data_label = QLabel("No dataset selected")
        self.data_label.setStyleSheet("color: #aaa;")
        select_data_btn = StyledButton("Select Dataset")
        select_data_btn.clicked.connect(self.select_data)
        data_layout.addWidget(self.data_label)
        data_layout.addWidget(select_data_btn)
        file_layout.addWidget(data_widget)

        # Model selection
        model_widget = QWidget()
        model_layout = QHBoxLayout(model_widget)
        self.model_label = QLabel("No model selected")
        self.model_label.setStyleSheet("color: #aaa;")
        select_model_btn = StyledButton("Select Model")
        select_model_btn.clicked.connect(self.select_model)
        model_layout.addWidget(self.model_label)
        model_layout.addWidget(select_model_btn)
        file_layout.addWidget(model_widget)
        
        controls_layout.addWidget(file_section)
        
        # Options section
        options_section = QFrame()
        options_section.setFrameStyle(QFrame.Shape.StyledPanel | QFrame.Shadow.Raised)
        options_layout = QVBoxLayout(options_section)
        
        # Add instance count selector
        instance_widget = QWidget()
        instance_layout = QHBoxLayout(instance_widget)
        instance_label = QLabel("Number of instances to analyze:")
        self.instance_combo = QComboBox()
        self.instance_combo.addItems(['1', '3', '5', '10'])
        self.instance_combo.setCurrentText('5')
        instance_layout.addWidget(instance_label)
        instance_layout.addWidget(self.instance_combo)
        options_layout.addWidget(instance_widget)
        
        controls_layout.addWidget(options_section)
        
        # Generate button
        self.generate_btn = StyledButton("Generate LIME Report")
        self.generate_btn.setEnabled(False)
        self.generate_btn.clicked.connect(self.generate_report)
        controls_layout.addWidget(self.generate_btn)
        
        # Progress bar
        self.progress_bar = StyledProgressBar()
        self.progress_bar.setVisible(False)
        controls_layout.addWidget(self.progress_bar)
        
        # Status log
        self.log_output = QTextEdit()
        self.log_output.setReadOnly(True)
        self.log_output.setStyleSheet("""
            QTextEdit {
                background-color: #2d2d2d;
                color: #fff;
                border: 1px solid #444;
                border-radius: 4px;
                padding: 5px;
            }
        """)
        self.log_output.setMinimumHeight(300)
        controls_layout.addWidget(self.log_output)
        
        splitter.addWidget(controls_widget)
        
        # Set splitter sizes
        splitter.setSizes([int(self.width() * 0.6), int(self.width() * 0.4)])
        
        # Add splitter to main layout
        main_layout.addWidget(splitter)
        
        self.setCentralWidget(main_widget)

    def setup_table_view(self, table_view):
        table_view.setStyleSheet("""
            QTableView {
                background-color: #2d2d2d;
                alternate-background-color: #353535;
                color: #fff;
                gridline-color: #444;
                selection-background-color: #4CAF50;
                selection-color: #fff;
            }
            QHeaderView::section {
                background-color: #404040;
                color: #fff;
                padding: 5px;
                border: 1px solid #444;
            }
        """)
        table_view.setAlternatingRowColors(True)
        table_view.horizontalHeader().setStretchLastSection(True)
        table_view.verticalHeader().setDefaultSectionSize(25)
        table_view.setShowGrid(True)

    def setup_stats_label(self, label):
        label.setStyleSheet("""
            QLabel {
                background-color: #2d2d2d;
                color: #fff;
                padding: 10px;
                border: 1px solid #444;
                border-radius: 4px;
                margin-top: 5px;
            }
        """)

    def update_stats_labels(self, original_df, processed_df):
        # Update statistics for original data
        original_stats = (
            f"Original Data Statistics:\n"
            f"Rows: {len(original_df)}\n"
            f"Columns: {len(original_df.columns)}\n"
            f"Missing Values: {original_df.isnull().sum().sum()}"
        )
        self.original_stats_label.setText(original_stats)
        
        # Update statistics for processed data
        processed_stats = (
            f"Processed Data Statistics:\n"
            f"Rows: {len(processed_df)}\n"
            f"Columns: {len(processed_df.columns)}\n"
            f"Missing Values: {processed_df.isnull().sum().sum()}"
        )
        self.processed_stats_label.setText(processed_stats)

    def select_data(self):
        file_path, _ = QFileDialog.getOpenFileName(
            self, "Select Dataset", "", "CSV Files (*.csv)"
        )
        if file_path:
            self.data_path = file_path
            self.data_label.setText(f"Dataset: {os.path.basename(file_path)}")
            
            try:
                # Load original data
                original_df = pd.read_csv(file_path)
                self.original_table_model.load_data(original_df)
                
                # Apply preprocessing pipeline
                pipeline = DataPreprocessingPipeline()
                processed_df = pipeline.transform(original_df)
                self.processed_table_model.load_data(processed_df)
                
                # Update statistics
                self.update_stats_labels(original_df, processed_df)
                
                self.log_output.append(f"Loaded and preprocessed dataset: {os.path.basename(file_path)}")
                self.check_generate_button()
                
                # Switch to processed data tab
                self.tab_widget.setCurrentIndex(1)
                
            except Exception as e:
                self.log_output.append(f"Error loading dataset: {str(e)}")

    def select_model(self):
        file_path, _ = QFileDialog.getOpenFileName(
            self, "Select Model", "", "Keras Model (*.keras);;All Files (*.*)"
        )
        if file_path:
            self.model_path = os.path.normpath(file_path.replace('/', os.sep))
            self.model_label.setText(f"Model: {os.path.basename(file_path)}")
            
            if os.path.exists(self.model_path):
                self.log_output.append(f"Selected model: {os.path.basename(file_path)}")
            else:
                self.log_output.append(f"Error: Model file not found")
            
            self.check_generate_button()

    def check_generate_button(self):
        self.generate_btn.setEnabled(self.data_path is not None and self.model_path is not None)

    def generate_report(self):
        if not self.data_path or not self.model_path:
            return
            
        self.progress_bar.setVisible(True)
        self.progress_bar.setValue(0)
        self.generate_btn.setEnabled(False)
        self.log_output.append("Starting report generation...")
        
        # Get number of instances from combo box
        num_instances = int(self.instance_combo.currentText())
        
        # Create and start the thread with number of instances
        self.thread = LIMEReportThread(self.data_path, self.model_path, num_instances)
        self.thread.progress.connect(self.update_progress)
        self.thread.status.connect(self.update_status)
        self.thread.finished.connect(self.report_finished)
        self.thread.error.connect(self.handle_error)
        self.thread.start()

    def handle_error(self, error_message):
        self.progress_bar.setVisible(False)
        self.generate_btn.setEnabled(True)
        self.log_output.append(f"Error: {error_message}")
        
        # Show error in more visible way
        error_msg = QLabel(error_message)
        error_msg.setStyleSheet("""
            QLabel {
                color: #ff4444;
                padding: 10px;
                background: #2d2d2d;
                border: 1px solid #ff4444;
                border-radius: 4px;
            }
        """)
        self.log_output.append("\n" + error_message)

    def update_progress(self, value):
        self.progress_bar.setValue(value)

    def update_status(self, message):
        self.log_output.append(message)

    def report_finished(self, results_dir):
        self.progress_bar.setVisible(False)
        self.generate_btn.setEnabled(True)
        self.log_output.append(f"Reports generated successfully in: {results_dir}")
        # Open the batch directory containing all vehicle reports
        os.startfile(results_dir)

In [7]:
app = QApplication(sys.argv)
window = MainWindow()
window.show()
sys.exit(app.exec())

1/1 [==============================] - 0s 83ms/step


SystemExit: 0